<a href="https://colab.research.google.com/github/margaridagomes/dataeng-basic-course/blob/main/spark_streaming/coinbase_consumer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Consumer Coinbase

- To be executed in Google Colab
- Connect to GCLOUD
- Read data from GCS as streaming
- Analyze data

In [ ]:
!pip install pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("GCS Streaming") \
    .config("spark.jars.packages", "com.google.cloud.bigdataoss:gcs-connector:hadoop3-2.2.5") \
    .getOrCreate()

In [ ]:
from google.colab import auth
auth.authenticate_user()

project_id = 'data-eng-dev-437916'
!gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
!apt-get install openjdk-11-jdk -y
!pip install pyspark gcsfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-11-jdk is already the newest version (11.0.27+6~us1-0ubuntu1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
from pyspark.sql import SparkSession

GCS_JAR = "https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.9/gcs-connector-hadoop3-2.2.9-shaded.jar"

spark = SparkSession.builder \
    .appName("GCSStreamingDemo") \
    .config("spark.jars", GCS_JAR) \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
    .getOrCreate()

In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Inicializa a sessão Spark
spark = SparkSession.builder.appName("BTCPriceBatch").getOrCreate()

# Define o schema
schema = "type STRING, sequence LONG, product_id STRING, price STRING, time STRING"

# Lê os dados JSON em modo batch
df = spark.read.schema(schema).json("gs://edit-data-eng-dev/datalake/landing/btc/")

# Seleciona e transforma as colunas necessárias
transformed_df = df.select("time", "product_id", col("price").cast("double"))

# Grava o resultado como uma tabela temporária em memória (apenas para consulta)
transformed_df.createOrReplaceTempView("btc_price_batch")

In [29]:
transformed_df.show(10, False)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

schema = "type STRING, sequence LONG, product_id STRING, price STRING, time STRING"

dfs = spark.readStream.schema(schema).json("gs://edit-data-eng-dev/datalake/landing/btc/")

stream = dfs.select("time", "product_id", col("price").cast("double")) \
  .writeStream \
  .outputMode("append") \
  .queryName("btc_price_stream3") \
  .format("memory") \
  .start()

In [30]:
dfs = spark.sql("select * from btc_price_stream3")
dfs.show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `btc_price_stream3` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [btc_price_stream3], [], false


In [21]:
dfs.isStreaming

True

In [ ]:
dfs

In [20]:
df = spark.sql("select * from btc_price_stream")
df.show()

+----+----------+-----+
|time|product_id|price|
+----+----------+-----+
+----+----------+-----+



In [32]:
spark.sql("select * from btc_price_stream").show()

+--------------------+----------+---------+
|                time|product_id|    price|
+--------------------+----------+---------+
|2025-07-11T21:59:...|   BTC-USD|117695.26|
|2025-07-05T10:20:...|   BTC-USD|108163.06|
|2025-07-05T10:21:...|   BTC-USD|108163.06|
|2025-07-05T10:21:...|   BTC-USD|108163.06|
|2025-07-05T10:20:...|   BTC-USD|108163.06|
|2025-07-05T10:21:...|   BTC-USD|108163.06|
|2025-07-05T10:20:...|   BTC-USD|108163.06|
|2025-07-05T10:21:...|   BTC-USD|108163.06|
|2025-07-05T10:21:...|   BTC-USD|108163.06|
|2025-07-05T10:21:...|   BTC-USD|108163.06|
|2025-07-05T10:21:...|   BTC-USD|108163.06|
|2025-07-05T10:21:...|   BTC-USD|108163.06|
|2025-07-05T10:21:...|   BTC-USD|108156.85|
|2025-07-05T10:21:...|   BTC-USD|108156.85|
|2025-07-05T10:21:...|   BTC-USD|108156.85|
|2025-07-05T10:21:...|   BTC-USD|108156.85|
|2025-07-05T10:21:...|   BTC-USD|108156.85|
|2025-07-05T10:22:...|   BTC-USD|108156.85|
|2025-07-05T10:22:...|   BTC-USD|108171.17|
|2025-07-05T10:22:...|   BTC-USD

In [1]:
# 1. Instalar PySpark
!pip install pyspark -q

# 2. Autenticar com GCP
from google.colab import auth
auth.authenticate_user()

# 3. Definir projeto GCP
project_id = "data-eng-dev-437916"
!gcloud config set project {project_id}

# 4. Download do GCS connector
!mkdir -p /content/jars
!wget -q https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.9/gcs-connector-hadoop3-2.2.9-shaded.jar -O /content/jars/gcs-connector.jar

# 5. SparkSession com suporte a GCS
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder \
    .appName("GCSStreamingDemo") \
    .config("spark.jars", "/content/jars/gcs-connector.jar") \
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
    .getOrCreate()

# 6. Definir schema dos dados
schema = "type STRING, sequence LONG, product_id STRING, price STRING, time STRING"

# 7. Ler dados em streaming do GCS
dfs = spark.readStream.schema(schema).json("gs://edit-data-eng-dev/datalake/landing/btc/")

# 8. Processar e armazenar em memória para testes
stream = dfs.select("time", "product_id", col("price").cast("double")) \
  .writeStream \
  .outputMode("append") \
  .queryName("btc_price_stream") \
  .format("memory") \
  .start()

# 9. Mostrar resultados (esperar alguns segundos antes de executar)
import time
time.sleep(10)  # aguarda alguns dados serem lidos

spark.sql("SELECT * FROM btc_price_stream").show(10, False)


Updated property [core/project].
+----+----------+-----+
|time|product_id|price|
+----+----------+-----+
+----+----------+-----+



In [27]:
!pip install gcsfs -q

import gcsfs
fs = gcsfs.GCSFileSystem(project='data-eng-dev-437916')

# Lista ficheiros no path
fs.ls('edit-data-eng-dev/datalake/landing/btc')

['edit-data-eng-dev/datalake/landing/btc/btc_20250705102018542731.json',
 'edit-data-eng-dev/datalake/landing/btc/btc_20250705102028868336.json',
 'edit-data-eng-dev/datalake/landing/btc/btc_20250705102038992676.json',
 'edit-data-eng-dev/datalake/landing/btc/btc_20250705102049618668.json',
 'edit-data-eng-dev/datalake/landing/btc/btc_20250705102059337980.json',
 'edit-data-eng-dev/datalake/landing/btc/btc_20250705102109064736.json',
 'edit-data-eng-dev/datalake/landing/btc/btc_20250705102113334700.json',
 'edit-data-eng-dev/datalake/landing/btc/btc_20250705102118686132.json',
 'edit-data-eng-dev/datalake/landing/btc/btc_20250705102122960477.json',
 'edit-data-eng-dev/datalake/landing/btc/btc_20250705102128309577.json',
 'edit-data-eng-dev/datalake/landing/btc/btc_20250705102131880148.json',
 'edit-data-eng-dev/datalake/landing/btc/btc_20250705102138634808.json',
 'edit-data-eng-dev/datalake/landing/btc/btc_20250705102140800804.json',
 'edit-data-eng-dev/datalake/landing/btc/btc_202507

In [ ]:
spark.sql("SELECT * FROM btc_price_stream").show(10, False)

In [ ]:
# Analysis

# Latest Bitcoin price
# Calculate average BTC price per minute
# Calculate standard deviation of price over time
# How many price tickets per minute?
# Find anomalies (price == nulls or with strange values)

In [33]:
# Latest Bitcoin price

from pyspark.sql.functions import *

# Latest Bitcoin price
df = spark.sql("select * from btc_price_stream where product_id = 'BTC-USD' order by time desc limit 1")
df.show()

import pyspark.sql.functions as f
from pyspark.sql.window import Window
WindowSpec = Window.partitionBy("product_id").orderBy("time")
df2 = df.filter("product_id = 'BTC-USD'").withColumn("last_price", f.first("price").over(WindowSpec))
#df2 = df.filter(f.col("last_price")==f.col("price"))

# Calcula o primeiro preço por produto
#df2 = df.withColumn("last_price", f.first("price").over(window_spec))

# Filtra onde o preço atual é igual ao primeiro preço
#df2 = df2.filter(f.col("last_price") == f.col("price"))

+--------------------+----------+---------+
|                time|product_id|    price|
+--------------------+----------+---------+
|2025-07-11T21:59:...|   BTC-USD|117695.27|
+--------------------+----------+---------+



In [34]:
# Latest Bitcoin price
df2.show(10,False)

+---------------------------+----------+---------+----------+
|time                       |product_id|price    |last_price|
+---------------------------+----------+---------+----------+
|2025-07-11T21:59:48.480300Z|BTC-USD   |117695.27|117695.27 |
+---------------------------+----------+---------+----------+



In [35]:
# count and average per product_id
df.groupBy("product_id").agg(count(lit("1")).alias("count"), avg("price").alias("avg_price")).show()

+----------+-----+---------+
|product_id|count|avg_price|
+----------+-----+---------+
|   BTC-USD|    1|117695.27|
+----------+-----+---------+



In [36]:
!gsutil ls gs://edit-data-eng-dev/datalake/landing/btc/ | wc

   1558    1558  115292


In [37]:
df.select(f.min("time"), f.max("time")).show(1,False)

+---------------------------+---------------------------+
|min(time)                  |max(time)                  |
+---------------------------+---------------------------+
|2025-07-11T21:59:48.480300Z|2025-07-11T21:59:48.480300Z|
+---------------------------+---------------------------+



In [38]:
df3 = spark.read.json("gs://edit-data-eng-dev/datalake/landing/btc/btc_20250705102018542731.json")
df3.select(f.min("time"), f.max("time")).show(1,False)

+---------------------------+---------------------------+
|min(time)                  |max(time)                  |
+---------------------------+---------------------------+
|2025-07-05T10:20:17.674573Z|2025-07-05T10:20:17.674573Z|
+---------------------------+---------------------------+



In [39]:
# Calculate average BTC price per minute
#df.filter("product_id == 'BTC-USD'").groupBy(window("time","1 min")).agg(avg("price").alias("avg_price")).orderBy("window").show(10, False)
df.filter("product_id == 'BTC-USD'").groupBy(window("time", "1 minute")).agg(avg("price")).orderBy("window").show(10, False)

AnalysisException: [CANNOT_PARSE_INTERVAL] Unable to parse '1 min'. Please ensure that the value provided is in a valid format for defining an interval. You can reference the documentation for the correct format. If the issue persists, please double check that the input value is not null or empty and try again.

In [ ]:
# Calculate standard deviation of price over time
df.groupBy("product_id").agg(F.stddev("price").alias("stddev_price")).show()

In [ ]:
# How many price tickets per minute?
#df.groupBy(window("time", "1 min"), "product_id").agg(count(lit("1")).alias("count")).orderBy(window("time", "1 min"), "product_id").show(10, False)
df.groupBy(window("time", "1 minute")).agg(count(lit("1")).alias("count")).orderBy("window").show(10, False)

In [ ]:
# Find anomalies (price == nulls or with strange values)
#df.filter("product_id == 'BTC_USD'").filter((col("price").isNUll()) | (col("price") >= 100000)).show()

df.filter("product_id == 'BTC-USD'").filter((col("price").isNull()) | (col("price") > lit(100000))).show()


In [ ]:
#landing para outro stage
#dar stop do streaming, em vez de escrever na query, escrevemos em parquet e o format em vez de memory escreve num path diferente